# Bab 19: Melatih dan Menyebarkan Model TensorFlow dalam Skala Besar

### 1. Pendahuluan

Bab 19 adalah bab yang sangat praktis, berfokus pada transisi dari melatih model *Machine Learning* di lingkungan penelitian (misalnya, Jupyter Notebook) ke menyebarkannya di lingkungan produksi. Ini mencakup berbagai alat dan praktik terbaik untuk pelatihan skala besar, *deployment*, dan pemantauan.

Bab ini akan membahas:
* **Distribusi Pelatihan:** Melatih model di banyak perangkat (GPU/TPU) atau di banyak mesin.
* **TensorFlow Serving:** Menyebarkan model terlatih untuk inferensi.
* **TensorFlow Lite:** Mengoptimalkan model untuk perangkat seluler dan *edge*.
* **TensorFlow.js:** Menjalankan model di browser.

---

### 2. Pelatihan dalam Skala Besar (*Training at Scale*)

Melatih model *Deep Learning* yang besar pada dataset yang sangat besar seringkali memerlukan lebih dari satu mesin atau GPU.

#### a. Paralelisme Data vs Paralelisme Model
* **Paralelisme Data:** Setiap perangkat (GPU/CPU/TPU) menerima salinan model yang sama dan melatihnya pada *subset* data yang berbeda secara paralel. Gradien kemudian diagregasi untuk memperbarui bobot model.
* **Paralelisme Model:** Model dipecah menjadi beberapa bagian, dan setiap bagian dijalankan pada perangkat yang berbeda.

TensorFlow menyediakan `tf.distribute.Strategy` API untuk mempermudah pelatihan terdistribusi. `MirroredStrategy` adalah contoh yang baik untuk paralelisme data pada beberapa GPU di satu mesin.


In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

# Contoh penggunaan MirroredStrategy (konseptual)
# distribution = tf.distribute.MirroredStrategy()
#
# with distribution.scope():
#     # Semua pembuatan model dan kompilasi harus berada di dalam scope strategi
#     mirrored_model = keras.models.Sequential([...])
#     mirrored_model.compile(loss="mse", optimizer="sgd")
#
# # batch_size harus dapat dibagi rata dengan jumlah replika/GPU
# # history = mirrored_model.fit(X_train, y_train, epochs=10,
# #                                batch_size=32 * distribution.num_replicas_in_sync)
print("Kerangka untuk MirroredStrategy telah disiapkan.")

Kerangka untuk MirroredStrategy telah disiapkan.


---

### 3. TensorFlow Serving
Setelah model dilatih, Anda perlu cara untuk menyebarkannya agar aplikasi lain dapat menggunakannya untuk inferensi. TensorFlow Serving adalah sistem penyajian (serving system) yang fleksibel dan berkinerja tinggi untuk model machine learning, yang dirancang untuk lingkungan produksi.

Prosesnya:
1. Simpan Model: Simpan model terlatih Anda dalam format SavedModel TensorFlow. Ini adalah format universal yang berisi semua informasi yang diperlukan untuk menyajikan model.
2. Jalankan TF Serving: TF Serving dapat dijalankan menggunakan Docker. Anda cukup mengarahkan TF Serving ke direktori tempat SavedModel Anda berada.
3. Kirim Permintaan: Aplikasi klien dapat mengirim permintaan inferensi ke TF Serving melalui gRPC atau REST API.

In [2]:
# 1. Menyimpan model ke format SavedModel
# (Asumsikan 'model' adalah model Keras yang sudah dilatih)
# model.save("my_fashion_mnist_model", save_format="tf")

# 2. Perintah Docker untuk menjalankan TF Serving (dijalankan di terminal)
# docker run -it --rm -p 8500:8500 -p 8501:8501 \\
#    -v "$PWD/my_fashion_mnist_model:/models/my_fashion_mnist_model" \\
#    -e MODEL_NAME=my_fashion_mnist_model \\
#    tensorflow/serving

# 3. Mengirim permintaan dari klien Python (contoh)
import json
import requests

# X_new = X_test[:3] # Contoh data input
# request_json = json.dumps({"signature_name": "serving_default",
#                            "instances": X_new.tolist()})
# response = requests.post("http://localhost:8501/v1/models/my_fashion_mnist_model:predict",
#                          data=request_json)
# y_proba = json.loads(response.text)['predictions']
print("Kerangka untuk deployment dengan TF Serving telah disiapkan.")

Kerangka untuk deployment dengan TF Serving telah disiapkan.


---

### 4. TensorFlow Lite (TFLite)
Untuk men-deploy model di perangkat seluler atau embedded system, kita memerlukan model yang ringan dan cepat. TensorFlow Lite (TFLite) adalah toolkit untuk mengonversi dan mengoptimalkan model TensorFlow untuk inferensi di perangkat tersebut.

Prosesnya:
1. Konversi Model: Gunakan TFLiteConverter untuk mengonversi SavedModel atau model Keras menjadi format .tflite yang datar dan efisien.
2. Optimisasi (Kuantisasi): Selama konversi, Anda dapat menerapkan teknik optimisasi seperti kuantisasi, yang mengurangi presisi bobot model (misalnya, dari float32 ke int8). Ini secara signifikan mengurangi ukuran model dan mempercepat inferensi, dengan sedikit atau tanpa kehilangan akurasi.
3. Jalankan Inferensi: Gunakan interpreter TFLite di aplikasi seluler (Android/iOS) atau perangkat Linux (seperti Raspberry Pi) untuk menjalankan model .tflite.

In [3]:
# Mengonversi model Keras ke format TFLite
# converter = tf.lite.TFLiteConverter.from_keras_model(model)
# tflite_model = converter.convert()

# Menyimpan model .tflite
# with open("my_converted_model.tflite", "wb") as f:
#     f.write(tflite_model)

# Menggunakan kuantisasi untuk optimisasi
# converter_quant = tf.lite.TFLiteConverter.from_keras_model(model)
# converter_quant.optimizations = [tf.lite.Optimize.DEFAULT] # Mengaktifkan kuantisasi
# tflite_quantized_model = converter_quant.convert()
print("Kerangka untuk konversi ke TensorFlow Lite telah disiapkan.")

Kerangka untuk konversi ke TensorFlow Lite telah disiapkan.


### 5. TensorFlow.js (TFJS)
TensorFlow.js (TFJS) adalah library JavaScript untuk melatih dan men-deploy model Machine Learning langsung di browser atau di Node.js.

Manfaat:

* Tidak perlu instalasi di sisi pengguna.
* Interaktivitas tinggi.
* Privasi data (data tidak perlu meninggalkan browser pengguna).

Prosesnya:
1. Konversi Model: Gunakan command-line tool `tensorflowjs_converter` untuk mengubah `SavedModel` atau model Keras menjadi format yang dapat dibaca oleh TFJS.
2. Sajikan di Web: Muat dan jalankan model di aplikasi web Anda menggunakan library TFJS

```bash
# Perintah konversi (dijalankan di terminal)
# tensorflowjs_converter --input_format=keras \\
#                        ./my_keras_model.h5 ./my_tfjs_model
```